In [2]:
import pandas as pd

df=pd.read_csv("salaries.csv")
print(df.shape)
df.head()

(16, 4)


,company,job,degree,salary_more_then_100k
0,google,sales executive,bachelors,0
1,google,sales executive,masters,0
2,google,business manager,bachelors,1
3,google,business manager,masters,1
4,google,computer programmer,bachelors,0


### Feature Engineering - Label Encoding


In [4]:
df["degree_labels"]=df["degree"].map({
    'bachelors':1,
    'masters':2
})

In [ ]:
df.drop("degree",axis=1,inplace=True)

In [8]:
df.head()

,company,job,salary_more_then_100k,degree_labels
0,google,sales executive,0,1
1,google,sales executive,0,2
2,google,business manager,1,1
3,google,business manager,1,2
4,google,computer programmer,0,1


### Feature Engineering - One Hot Encoding

In [10]:
df_encoded = pd.get_dummies(df,columns=["company","job"],drop_first=True)

In [12]:
df_encoded.head()

,salary_more_then_100k,degree_labels,company_facebook,company_google,job_computer programmer,job_sales executive
0,0,1,False,True,False,True
1,0,2,False,True,False,True
2,1,1,False,True,False,False
3,1,2,False,True,False,False
4,0,1,False,True,True,False


### Model Training

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
X=df_encoded.drop("salary_more_then_100k",axis=1)
y=df_encoded["salary_more_then_100k"]

model = DecisionTreeClassifier(criterion='gini')
model.fit(X,y)
y_pred=model.predict(X)

report = classification_report(y,y_pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16



In [16]:
model = DecisionTreeClassifier(criterion='entropy')
model.fit(X,y)
y_pred=model.predict(X)

report = classification_report(y,y_pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16



In [19]:
from sklearn.tree import export_text

print(export_text(model,feature_names=list(X.columns)))

|--- company_facebook <= 0.50
|   |--- job_sales executive <= 0.50
|   |   |--- degree_labels <= 1.50
|   |   |   |--- company_google <= 0.50
|   |   |   |   |--- class: 0
|   |   |   |--- company_google >  0.50
|   |   |   |   |--- job_computer programmer <= 0.50
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- job_computer programmer >  0.50
|   |   |   |   |   |--- class: 0
|   |   |--- degree_labels >  1.50
|   |   |   |--- class: 1
|   |--- job_sales executive >  0.50
|   |   |--- class: 0
|--- company_facebook >  0.50
|   |--- class: 1



In [20]:
X.columns

Index(['degree_labels', 'company_facebook', 'company_google',
       'job_computer programmer', 'job_sales executive'],
      dtype='object')